# Assignment 3

## Hyperparameter Tunning

### Zifeng Xu, Wendi Chu, Kefan Yu

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
random.seed(8649)

## 6. Data Preparation (7 Points)

### 6.1 Load the dataset and display the dataframe (2 Points).

In [20]:
# load the dataset.
df=pd.read_csv("online_shoppers_intention.csv")

# change the data type of the column
df["Weekend"] = df["Weekend"].astype(int)
df["Revenue"] = df["Revenue"].astype(int)

# convert the categorical values of the column VisitorType into a numerical representation
df.insert(loc=16, column="VisitorTypeNumeric", value=pd.factorize(df['VisitorType'])[0] + 1)
df = df.drop('VisitorType', 1)

# Define a prefix
prefix = 'g06_'

# Add the prefix to all column names
df.columns = [prefix + col for col in df.columns]

# display the dataframe
df.head()

,g06_Administrative,g06_Administrative_Duration,g06_Informational,g06_Informational_Duration,g06_ProductRelated,g06_ProductRelated_Duration,g06_BounceRates,g06_ExitRates,g06_PageValues,g06_SpecialDay,g06_Month,g06_OperatingSystems,g06_Browser,g06_Region,g06_TrafficType,g06_VisitorTypeNumeric,g06_Weekend,g06_Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,1,0,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,1,0,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,1,0,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,1,0,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,1,1,0


### 6.2 Use describe to provide statistics on the pandas Dataframe (2 Points).

In [13]:
df.describe()

,g06_Administrative,g06_Administrative_Duration,g06_Informational,g06_Informational_Duration,g06_ProductRelated,g06_ProductRelated_Duration,g06_BounceRates,g06_ExitRates,g06_PageValues,g06_SpecialDay,g06_OperatingSystems,g06_Browser,g06_Region,g06_TrafficType,g06_VisitorTypeNumeric,g06_Weekend,g06_Revenue
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586,1.151176,0.232603,0.154745
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169,0.376989,0.422509,0.361676
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000,1.000000,0.000000,0.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000,1.000000,0.000000,0.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000,3.000000,1.000000,1.000000


### 6.3 Split the dataset into a Training set and a Test set. Justify your preferred split (3 Points)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [23]:
X = df.loc[:, df.columns != 'g06_Revenue'].to_numpy()
y = df.iloc[:,-1:].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=8649)

## 7. Classification Routine (12 Points)

### 7.1 RandomForestClassifier()

### 7.2 BaggingClassifier()

### 7.3 XGboostclassifier()

## 8. Classification with GridSearchCV (8 Points):